## Oxford AI Summit: Group 4 Submission

## Business Problem

#TODO

## Our Solution

#TODO

## Value

#TODO

# Phase 1: Demo

## Load the data

## Data Augmentation

## Train test split

## Train the model

## Exaluation

# Phase 2: Sexy Graph Database Search Solution

#TODO